<a href="https://colab.research.google.com/github/txusser/Master_IA_Sanidad/blob/main/Modulo_3/NLP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalamos algunas dependencias.

In [19]:
!mkdir tutorial
!git clone https://github.com/scikit-learn/scikit-learn.git
!pip install scikit-learn
!cp -r scikit-learn/doc/tutorial/text_analytics/ tutorial
!python tutorial/text_analytics/data/twenty_newsgroups/fetch_data.py

mkdir: cannot create directory ‘tutorial’: File exists
fatal: destination path 'scikit-learn' already exists and is not an empty directory.


Nos descargamos el dataset que vamos a utilizar en este ejemplo. 
Es un dataset de e-mails (aproximadamente 12.000 documentos en total), divididos (casi) uniformemente en 20 grupos de dependiendo de los temas que se tratan. Fue recopilado originalmente por Ken Lang, probablemente para su artículo "Newsweeder: Learning to filter netnews".

In [23]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train')
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [22]:
len(twenty_train.data)

11314

In [28]:
for t in twenty_train.target[:10]:
  print(twenty_train.target_names[t])

rec.autos
comp.sys.mac.hardware
comp.sys.mac.hardware
comp.graphics
sci.space
talk.politics.guns
sci.med
comp.sys.ibm.pc.hardware
comp.os.ms-windows.misc
comp.sys.mac.hardware


In [34]:
print("\n".join(twenty_train.data[4].split("\n")))

From: jcm@head-cfa.harvard.edu (Jonathan McDowell)
Subject: Re: Shuttle Launch Question
Organization: Smithsonian Astrophysical Observatory, Cambridge, MA,  USA
Distribution: sci
Lines: 23

From article <C5owCB.n3p@world.std.com>, by tombaker@world.std.com (Tom A Baker):
>>In article <C5JLwx.4H9.1@cs.cmu.edu>, ETRAT@ttacs1.ttu.edu (Pack Rat) writes...
>>>"Clear caution & warning memory.  Verify no unexpected
>>>errors. ...".  I am wondering what an "expected error" might
>>>be.  Sorry if this is a really dumb question, but
> 
> Parity errors in memory or previously known conditions that were waivered.
>    "Yes that is an error, but we already knew about it"
> I'd be curious as to what the real meaning of the quote is.
> 
> tom


My understanding is that the 'expected errors' are basically
known bugs in the warning system software - things are checked
that don't have the right values in yet because they aren't
set till after launch, and suchlike. Rather than fix the code
and possibly i

‎El preprocesamiento de texto, la tokenización y el filtrado de palabras clave se incluyen en ‎‎CountVectorizer‎‎, que crea un diccionario de características y transforma los emails en vectores de características:.



In [35]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

Veamos el tamaño de nuestro vocabulario

In [36]:
count_vect.vocabulary_.get(u'algorithm')

27366

‎El recuento de ocurrencias es un buen comienzo, pero hay un problema: los documentos más largos tendrán valores de recuento promedio más altos que los documentos más cortos, aunque puedan hablar sobre los mismos temas.‎

‎Para evitar estas posibles discrepancias basta con dividir el número de apariciones de cada palabra en un documento por el número total de palabras en el documento: estas nuevas características se denominan Frecuencias de Término (tf).

‎Otro refinamiento además de transformar los valores en tf es reducir los pesos de las palabras que aparecen en muchos documentos del corpus y, por lo tanto, son menos informativas que las que ocurren solo en una porción más pequeña del corpus.‎

‎Esta reducción de escala se denomina ‎‎tf-idf‎‎ para "Term Frequency times Inverse Document Frequency".‎

‎Tanto ‎‎tf‎‎ como ‎‎tf-idf‎‎ se pueden calcular de la siguiente manera utilizando ‎‎TfidfTransformer‎‎:‎

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

‎En el código de ejemplo anterior, en primer lugar usamos el método "*fit*" para ajustar nuestro estimador a los datos y, en segundo lugar, el método "*transform*" para transformar nuestra matriz de conteo en una representación tf-idf. Estos dos pasos se pueden combinar para lograr el mismo resultado final al omitir el procesamiento redundante. Esto se hace mediante el uso del método "*fit_transform*". Ejercicio: Reemplazar por el método "*fit_transform*"

In [43]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

‎Ahora que tenemos nuestras features, podemos entrenar un clasificador para que intente predecir la categoría de un post. Comencemos con un clasificador ‎‎naive Bayes, que proporciona una buena línea de base para esta tarea. Existen varias variantes de este clasificador; la más adecuada para el recuento de palabras es la variante multinomial

In [45]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

Para tratar de predecir el resultado en un nuevo documento, necesitamos extraer las características utilizando casi la misma cadena de extracción de características que antes. La diferencia es que llamamos a "*transform*" en lugar de a "*fit_transform*", ya que ya se han ajustado al conjunto de entrenamiento

In [49]:
docs_new = ['Jesus Loves You. Follow his guidance and you will be able to reach heaven', 'Earth is 7 light years away from the sun','You need to ensure that the engine temperature does not reach above 90 degrees']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
  print('%r => %s' % (doc, twenty_train.target_names[category]))

'Jesus Loves You. Follow his guidance and you will be able to reach heaven' => soc.religion.christian
'Earth is 7 light years away from the sun' => sci.space
'You need to ensure that the engine temperature does not reach above 90 degrees' => rec.autos
